In [2]:
import numpy as np
import pandas as pd


In [3]:
df=pd.read_csv('IMDB Dataset.csv')

In [4]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive


In [5]:
# Replace 'negative' with 0 and 'positive' with 1 in the 'sentiment' column
df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1})

C:\Users\HP\AppData\Local\Temp\ipykernel_72768\2167417402.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df['sentiment'] = df['sentiment'].replace({'negative': 0, 'positive': 1})


In [6]:
df.head()

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [7]:
df.shape

(50000, 2)

In [8]:
# Removing last 30 rows for manual testing
df_testing = df.head(30)
for i in range(0,30):
    df.drop([i], axis = 0, inplace = True)



In [9]:
df_testing.head(5)

,review,sentiment
0,One of the other reviewers has mentioned that ...,1
1,A wonderful little production. <br /><br />The...,1
2,I thought this was a wonderful way to spend ti...,1
3,Basically there's a family where a little boy ...,0
4,"Petter Mattei's ""Love in the Time of Money"" is...",1


In [10]:
df_testing.to_csv("testing.csv")

In [11]:
df.isnull().sum()

review       0
sentiment    0
dtype: int64

In [12]:
import re
import nltk
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()
from tqdm import tqdm

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\HP\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [13]:

tqdm.pandas(desc="Preprocessing text")  # Initialize progress bar

ps = PorterStemmer()
stop_words = set(stopwords.words('english'))
# Preprocessing function
def preprocess(text):
    # Keep only letters and lowercase
    text = re.sub('[^a-zA-Z]', ' ', text).lower().split()
    # Remove stopwords and stem
    words = [ps.stem(word) for word in text if word not in stop_words]
    return ' '.join(words)

# Apply to entire column with progress bar
df['review'] = df['review'].progress_apply(preprocess)



Preprocessing text: 100%|██████████| 49970/49970 [01:10<00:00, 710.55it/s]


In [14]:
x = df["review"]
y = df["sentiment"]

In [15]:
#train test split
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=0)

In [16]:
from sklearn.feature_extraction.text import TfidfVectorizer

vectorization = TfidfVectorizer()
xv_train = vectorization.fit_transform(X_train)
xv_test = vectorization.transform(X_test)

In [17]:
from sklearn.linear_model import LogisticRegression
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.ensemble import RandomForestClassifier


In [18]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [19]:
# Choose a model
models = {
  
    "Logistic": LogisticRegression(max_iter=200),
    "DecisionTree": DecisionTreeClassifier(),
    "RandomForest": RandomForestClassifier(n_estimators=100),
    "GradientBoosting": GradientBoostingClassifier(n_estimators=100),

}

In [20]:

for name, model in models.items():
    model.fit(xv_train, y_train)
    y_pred = model.predict(xv_test)
    print("=== Model:", name, "===")
    print("Accuracy:", accuracy_score(y_test, y_pred))


=== Model: Logistic ===
Accuracy: 0.8948369021412848
=== Model: DecisionTree ===
Accuracy: 0.7091254752851711
=== Model: RandomForest ===
Accuracy: 0.8547128276966179
=== Model: GradientBoosting ===
Accuracy: 0.8151891134680809


In [21]:
# Save the trained RandomForest model and vectorizer
import joblib

# Save RandomForest model
joblib.dump(models["RandomForest"], "random_forest_model.pkl")
joblib.dump(models["Logistic"], "logistic_model.pkl")
joblib.dump(models["DecisionTree"], "DecisionTree_model.pkl")
joblib.dump(models["GradientBoosting"], "GradientBoosting_model.pkl")


# Save TfidfVectorizer
joblib.dump(vectorization, "tfidf_vectorizer.pkl")

print("ALL Model and vectorizer saved")

ALL Model and vectorizer saved


In [22]:

# Load saved models and vectorizer
rf_model = joblib.load("logistic_model.pkl")
vectorizer = joblib.load("tfidf_vectorizer.pkl")

# Example: Predict on new news text
def predict_news(news_text):
    processed = preprocess(news_text)
    vect = vectorizer.transform([processed])
    pred = rf_model.predict(vect)
    return "negative" if pred[0] == 0 else "positive"

# Test with a sample news
sample_news = "Your news article text here"
print(predict_news(sample_news))

negative


In [23]:
# Batch prediction on testing.csv
import pandas as pd

test_df = pd.read_csv("testing.csv")

# Preprocess and predict
preds = []
for text in test_df['review']:
    processed = preprocess(str(text))
    vect = vectorizer.transform([processed])
    pred = rf_model.predict(vect)
    preds.append(pred[0])

test_df['predicted_class'] = preds
test_df['predicted_label'] = test_df['predicted_class'].apply(lambda x: 'negative' if x == 0 else 'positive')

test_df.to_csv("testing_with_predictions.csv", index=False)
print("Batch predictions saved to testing_with_predictions.csv")

Batch predictions saved to testing_with_predictions.csv


In [24]:
df1=pd.read_csv("testing_with_predictions.csv")
df1.head(30)

,Unnamed: 0,review,sentiment,predicted_class,predicted_label
0,0,One of the other reviewers has mentioned that ...,1,1,positive
1,1,A wonderful little production. <br /><br />The...,1,1,positive
2,2,I thought this was a wonderful way to spend ti...,1,1,positive
3,3,Basically there's a family where a little boy ...,0,0,negative
4,4,"Petter Mattei's ""Love in the Time of Money"" is...",1,1,positive
5,5,"Probably my all-time favorite movie, a story o...",1,1,positive
6,6,I sure would like to see a resurrection of a u...,1,0,negative
7,7,"This show was an amazing, fresh & innovative i...",0,0,negative
8,8,Encouraged by the positive comments about this...,0,0,negative
9,9,If you like original gut wrenching laughter yo...,1,1,positive
